# CS246 - Homework 1

## Question 1

### Spark

### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 281.3 MB 8.7 kB/s 
     |████████████████████████████████| 198 kB 41.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=961189cfe99e4a1aaf0b812038fb3f9fb98884154ef0281bb63f16e868689fcc
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 37 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Re

Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Get the data

In [ ]:
id='1XPgiA3NFj3V_7SOt9oDovHYixa_1NTOy'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('soc-LiveJournal1Adj.txt')

ApiRequestError: ignored

### question1 
数据格式为$<User><TAB><Friends>$ <br>
要求找到N = 10 个现在并不是User的好友，但是与User有最多的共同好友的用户。默认所有的好友都是互相关注的，即数据结构为无向图。要求找到top10的second-degree friends。

Import Spark

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd
import itertools

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

对于给出的$<User><TAB><Friends>$结构，我们可以将他分为两部分：只包含了user本身和其中一个friend的user_friends_pairs,和在friends中任取两个friend组合成的mutal_friends_pairs。显然，mutal_friends_pairs中既包含了已经互相关注的friend_pairs和还未互相关注的target_pairs。<br>
我们可以先行求出已经互为好友的user_friends_pairs，在mutal_friends_pairs中过滤掉这些user_friends_pairs，那么剩下的结果就是target_pairs。由于我们是在$<User><TAB><Friends>$下构造的mutal_friends_pairs，所以target_pairs一定是sencond-degree-friends。


In [ ]:
def generate_friends_list(line):
    line = line.split('\t')

    if len(line[1]) == 1 or line[0] == '':
        return []

    user = line[0]
    friends = line[1].split(',')

    user_friend_pairs = [((user, friend), 0) for friend in friends if user != '']
    
    mutual_friend_pairs = [(pair, 1) for pair in itertools.permutations(friends, 2)]

    return user_friend_pairs + mutual_friend_pairs

def recommend_friends(line):
    user = line[0]
    recommend_friend = list(line[1])

    recommend_friend_sort = sorted(recommend_friend, key = lambda friend: (-friend[1], int(friend[0])))

    return (user, [friend[0] for friend in recommend_friend_sort[:10]])


data = sc.textFile('soc-LiveJournal1Adj.txt')

user_friends = data.flatMap(lambda line: generate_friends_list(line))

already_friend = user_friends.filter(lambda x: x[1] == 0)

not_friend_yet = user_friends.subtractByKey(already_friend)

friends_recommended = not_friend_yet.reduceByKey(lambda x, y: x + y).\
            map(lambda x: (x[0][0], (x[0][1], x[1]))).\
            groupByKey().\
            map(recommend_friends)


我们现在已经求出了target_pairs。现在的问题是如何将这些target_pairs根据他们的共同好友数目进行排序。<br>
我们已经知道target_pairs是在某个user的好友列表中构造的，那么在target_pairs的集合中，相同的target_pais每出现一次，就意味着pairs中有一个共同好友，我们直接对于pairs进行计数操作，得到他们的共同好友数。<br>

target_pairs经过计数后的数据结构为(user1,user2,共同好友数)。我们更改数据结构为<br>
(user1,(user2,共同好友数))，对user1进行groupby操作，得到了关于user1的所有待推荐好友，再根据共同好友数和user2的id号码进行降序排序，就得到了user1的待推荐好友列表。question1完成求解。

In [5]:
friends_recommended.lookup('8941')

NameError: ignored

In [ ]:
ids = '924, 8941, 8942, 9019, 9020, 9021, 9022, 9990, 9992, 9993'.split(', ')
ids

['924', '8941', '8942', '9019', '9020', '9021', '9022', '9990', '9992', '9993']

In [ ]:
for id in ids:
    print(f'{id}:\t{",".join(friend for friend in friends_recommended.lookup(id)[0])}')

924:	439,2409,6995,11860,15416,43748,45881
8941:	8943,8944,8940
8942:	8939,8940,8943,8944
9019:	9022,317,9023
9020:	9021,9016,9017,9022,317,9023
9021:	9020,9016,9017,9022,317,9023
9022:	9019,9020,9021,317,9016,9017,9023
9990:	13134,13478,13877,34299,34485,34642,37941
9992:	9987,9989,35667,9991
9993:	9991,13134,13478,13877,34299,34485,34642,37941
